Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Use FLAML to Tune ChatGPT

FLAML offers a cost-effective hyperparameter optimization technique [EcoOptiGen](https://arxiv.org/abs/2303.04673) for tuning Large Language Models. Our study finds that tuning hyperparameters can significantly improve the utility of LLMs.

In this notebook, we tune OpenAI ChatGPT (both GPT-3.5 and GPT-4) models for math problem solving. We use [the MATH benchmark](https://crfm.stanford.edu/helm/latest/?group=math_chain_of_thought) for measuring mathematical problem solving on competition math problems with chain-of-thoughts style reasoning. 

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [openai,blendsearch] option:
```bash
pip install flaml[openai,blendsearch]==1.2.1
```

In [1]:
# %pip install flaml[openai,blendsearch]==1.2.1 datasets

Set your OpenAI key:

In [2]:
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "<your OpenAI API key here>"

Uncomment the following to use Azure OpenAI:

In [3]:
# import openai
# openai.api_type = "azure"
# openai.api_base = "https://<your_endpoint>.openai.azure.com/"
# openai.api_version = "2023-03-15-preview"

## Load dataset

First, we load the competition_math dataset. The dataset contains 201 "Level 2" Algebra examples. We use a random sample of 20 examples for tuning the generation hyperparameters and the remaining for evaluation.

In [4]:
import datasets

seed = 41
data = datasets.load_dataset("competition_math")
train_data = data["train"].shuffle(seed=seed)
test_data = data["test"].shuffle(seed=seed)
n_tune_data = 20
tune_data = [
    {
        "problem": train_data[x]["problem"],
        "solution": train_data[x]["solution"],
    }
    for x in range(len(train_data)) if train_data[x]["level"] == "Level 2" and train_data[x]["type"] == "Algebra"
][:n_tune_data]
test_data = [
    {
        "problem": test_data[x]["problem"],
        "solution": test_data[x]["solution"],
    }
    for x in range(len(test_data)) if test_data[x]["level"] == "Level 2" and test_data[x]["type"] == "Algebra"
]
print(len(tune_data), len(test_data))


Using custom data configuration default
Found cached dataset competition_math (/home/vscode/.cache/huggingface/datasets/competition_math/default/1.0.0/2a2a2995c2847186883ecd64f69be7d602b8a6f6b51950624d4dc2263f93333b)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/vscode/.cache/huggingface/datasets/competition_math/default/1.0.0/2a2a2995c2847186883ecd64f69be7d602b8a6f6b51950624d4dc2263f93333b/cache-f1cfe8228271b121.arrow
Loading cached shuffled indices for dataset at /home/vscode/.cache/huggingface/datasets/competition_math/default/1.0.0/2a2a2995c2847186883ecd64f69be7d602b8a6f6b51950624d4dc2263f93333b/cache-d155a2d38c23bd53.arrow


20 201


Check a tuning example:

In [5]:
print(tune_data[1]["problem"])

Rationalize the denominator of $\displaystyle\frac{21}{\sqrt{21}}$.


Here is one example of the canonical solution:

In [6]:
print(tune_data[1]["solution"])

$\dfrac{21}{\sqrt{21}} = \dfrac{21}{\sqrt{21}} \cdot \dfrac{\sqrt{21}}{\sqrt{21}} = \dfrac{21\sqrt{21}}{21} = \boxed{\!\sqrt{21}}$.


## Define Success Metric

Before we start tuning, we need to define the success metric we want to optimize. For each math task, we use voting to select a response with the most common answers out of all the generated responses. If it has an equivalent answer to the canonical solution, we consider the task as successfully solved. Then we can optimize the mean success rate of a collection of tasks.

In [7]:
from flaml.autogen.math_utils import eval_math_responses

## Use the tuning data to find a good configuration

### Import the oai and tune subpackages from flaml.

FLAML has provided an API for hyperparameter optimization of OpenAI ChatGPT models: `oai.ChatCompletion.tune` and to make a request with the tuned config: `oai.ChatCompletion.create`. First, we import oai from flaml:

In [8]:
from flaml import oai

For (local) reproducibility and cost efficiency, we cache responses from OpenAI.

In [9]:
oai.ChatCompletion.set_cache(seed)

This will create a disk cache in ".cache/{seed}". You can change `cache_path` in `set_cache()`. The cache for different seeds are stored separately.

### Perform tuning

The tuning will take a while to finish, depending on the optimization budget. The tuning will be performed under the specified optimization budgets.

* `inference_budget` is the target average inference budget per instance in the benchmark. For example, 0.004 means the target inference budget is 0.004 dollars, which translates to 2000 tokens (input + output combined) if the gpt-3.5-turbo model is used.
* `optimization_budget` is the total budget allowed to perform the tuning. For example, 1 means 1 dollars are allowed in total, which translates to 500K tokens for the gpt-3.5-turbo model.
* `num_sumples` is the number of different hyperparameter configurations which is allowed to try. The tuning will stop after either num_samples trials or after optimization_budget dollars spent, whichever happens first. -1 means no hard restriction in the number of trials and the actual number is decided by `optimization_budget`.

Users can specify tuning data, optimization metric, optimization mode, evaluation function, search spaces etc.. The default search space is:

```python
default_search_space = {
    "model": tune.choice([
        "gpt-3.5-turbo",
        "gpt-4",
    ]),
    "temperature_or_top_p": tune.choice(
        [
            {"temperature": tune.uniform(0, 1)},
            {"top_p": tune.uniform(0, 1)},
        ]
    ),
    "max_tokens": tune.lograndint(50, 1000),
    "n": tune.randint(1, 100),
    "prompt": "{prompt}",
}
```

The default search space can be overridden by users' input.
For example, the following code specifies a fixed prompt template. For hyperparameters which don't appear in users' input, the default search space will be used.

In [10]:
import logging

prompts = ["{problem} Solve the problem carefully. Simplify your answer as much as possible. Put the final answer in \\boxed{{}}."]
config, analysis = oai.ChatCompletion.tune(
    data=tune_data,  # the data for tuning
    metric="success_vote",  # the metric to optimize
    mode="max",  # the optimization mode
    eval_func=eval_math_responses,  # the evaluation function to return the success metrics
    # log_file_name="logs/math.log",  # the log file name
    inference_budget=0.02,  # the inference budget (dollar per instance)
    optimization_budget=1,  # the optimization budget (dollar in total)
    # num_samples can further limit the number of trials for different hyperparameter configurations;
    # -1 means decided by the optimization budget only
    num_samples=-1,
    # model="gpt-3.5-turbo",  # uncomment if you don't have access to gpt-4
    prompt=prompts,  # the prompt templates to choose from
    # stop="###",  # the stop sequence
    logging_level=logging.INFO,  # the logging level
)


[I 2023-04-02 21:11:48,741] A new study created in memory with name: optuna
[I 2023-04-02 21:11:48,744] A new study created in memory with name: optuna


[flaml.tune.tune: 04-02 21:11:48] {832} INFO - trial 1 config: {'prompt': 0, 'subspace': {'model': 'gpt-4', 'max_tokens': 347, 'temperature_or_top_p': {'temperature': 0.36865945026811975}, 'n': 1}}
[flaml.tune.tune: 04-02 21:11:48] {215} INFO - result: {'expected_success': 0.8, 'success': 0.8, 'success_vote': 0.8, 'voted_answer': 'We use the distance formula to find the distance between the points $(0,4)$ and $(3,0)$: $$\\sqrt{(3-0)^2+(0-4)^2}=\\sqrt{3^2+(-4)^2}=\\sqrt{9+16}=\\sqrt{25}=\\boxed{5}.$$', 'votes': 1.0, 'total_cost': 0.14595, 'cost': 0.14595, 'inference_cost': 0.007297499999999999, 'training_iteration': 0, 'config': {'prompt': 0, 'subspace': {'model': 'gpt-4', 'max_tokens': 347, 'temperature_or_top_p': {'temperature': 0.36865945026811975}, 'n': 1}}, 'config/prompt': 0, 'config/subspace': {'model': 'gpt-4', 'max_tokens': 347, 'temperature_or_top_p': {'temperature': 0.36865945026811975}, 'n': 1}, 'experiment_tag': 'exp', 'time_total_s': 0.005024433135986328}
[flaml.tune.tune:

### Output tuning results

After the tuning, we can print out the config and the result found by FLAML:

In [11]:
print("optimized config", config)
print("best result on tuning data", analysis.best_result)

optimized config {'prompt': '{problem} Solve the problem carefully. Simplify your answer as much as possible. Put the final answer in \\boxed{{}}.', 'max_tokens': 433, 'n': 54, 'model': 'gpt-3.5-turbo', 'stop': None, 'top_p': 0.7145757833976906}
best result on tuning data {'expected_success': 0.9999912490789106, 'success': 1.0, 'success_vote': 0.9, 'voted_answer': 'We use the distance formula: $$\\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}.$$ Letting $(x_1,y_1)=(0,4)$ and $(x_2,y_2)=(3,0)$, we have: \\begin{align*}\n\\sqrt{(3-0)^2+(0-4)^2} &= \\sqrt{9+16} \\\\\n&= \\sqrt{25} \\\\\n&= \\boxed{5}.\n\\end{align*}', 'votes': 41.65, 'total_cost': 0.6993100000000001, 'cost': 0.30776799999999993, 'inference_cost': 0.014812200000000001, 'training_iteration': 0, 'config': {'prompt': 0, 'subspace': {'max_tokens': 433, 'temperature_or_top_p': {'top_p': 0.7145757833976906}, 'n': 54, 'model': 'gpt-3.5-turbo'}}, 'config/prompt': 0, 'config/subspace': {'max_tokens': 433, 'temperature_or_top_p': {'top_p': 0.714575

### Make a request with the tuned config

We can apply the tuned config on the request for an example task:

In [12]:
response = oai.ChatCompletion.create(context=tune_data[1], **config)
metric_results = eval_math_responses(oai.ChatCompletion.extract_text(response), **tune_data[1])
print("response on an example data instance:", response)
print("metric_results on the example data instance:", metric_results)


response on an example data instance: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "We want to eliminate the radical in the denominator. We can do this by multiplying the numerator and denominator by $\\sqrt{21}$: $$\\frac{21}{\\sqrt{21}}=\\frac{21\\cdot\\sqrt{21}}{\\sqrt{21}\\cdot\\sqrt{21}}=\\frac{21\\sqrt{21}}{21}=\\sqrt{21}.$$Therefore, the simplified expression is $\\boxed{\\sqrt{21}}$.",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "message": {
        "content": "We have $\\displaystyle\\frac{21}{\\sqrt{21}}=\\frac{21}{\\sqrt{21}}\\cdot\\frac{\\sqrt{21}}{\\sqrt{21}}=\\frac{21\\sqrt{21}}{21}=\\boxed{\\sqrt{21}}$.",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 2,
      "message": {
        "content": "We can simplify the expression by multiplying both the numerator and the denominator by $\\sqrt{21}$: $$\\frac

### Evaluate the success rate on the test data

You can use flaml's `oai.ChatCompletion.test` to evaluate the performance of an entire dataset with the tuned config. The following code will take a while (30 mins to 1 hour) to evaluate all the test data instances if uncommented and run. It will cost roughly $3. 

In [13]:
# result = oai.Completion.test(test_data, config, eval_math_responses)
# print("performance on test data with the tuned config:", result)

performance on test data with the tuned config: {'expected_success': 0.9765082554311302, 'success': 0.9800995024875622, 'success_vote': 0.9104477611940298, 'votes': 39.87064676616915, 'cost': 3.0702499999999997, 'inference_cost': 0.015274875621890546}


What about the default, untuned gpt-4 config (with the same prompt as the tuned config)? We can evaluate it and compare:

In [14]:
# assuming you have access to gpt-4; otherwise use gpt-3.5-turbo
# the following code will cost roughly $2 if uncommented and run.

# default_config = {"model": 'gpt-4', "prompt": prompts[0]}
# default_result = oai.Completion.test(test_data, default_config, eval_math_responses)
# print("performance on test data from gpt-4 with a default config:", default_result)

performance on test data from gpt-4 with a default config: {'expected_success': 0.6716417910447762, 'success': 0.6716417910447762, 'success_vote': 0.6716417910447762, 'votes': 0.9900497512437811, 'cost': 1.92477, 'inference_cost': 0.009575970149253732}


In [15]:
# print("tuned config succeeds in {:.1f}% test cases".format(result["success_vote"] * 100))
# print("untuned config succeeds in {:.1f}% test cases".format(default_result["success_vote"] * 100))

tuned config succeeds in 91.0% test cases
untuned config succeeds in 67.2% test cases


The default use of GPT-4 has a much lower accuracy. Note that the default config has a lower inference cost. What if we heuristically increase the number of responses n?

In [16]:
# The following evaluation costs $3 and longer than one hour if you uncomment it and run it.

# config_n2 = {"model": 'gpt-4', "prompt": prompts[0], "n": 2}
# result_n2 = oai.ChatCompletion.test(test_data, config_n2, eval_math_responses)
# print("performance on test data from gpt-4 with a default config and n=2:", result_n2)


performance on test data from gpt-4 with a default config and n=2: {'expected_success': 0.7699004975124378, 'success': 0.835820895522388, 'success_vote': 0.7064676616915423, 'votes': 1.5870646766169154, 'cost': 3.267810000000001, 'inference_cost': 0.016257761194029857}


The inference cost is doubled and matches the tuned config. But the success rate doesn't improve much. What if we further increase the number of responses n to 5?

In [17]:
# The following evaluation costs $8 and longer than one hour if you uncomment it and run it.

# config_n5 = {"model": 'gpt-4', "prompt": prompts[0], "n": 5}
# result_n5 = oai.ChatCompletion.test(test_data, config_n5, eval_math_responses)
# print("performance on test data from gpt-4 with a default config and n=5:", result_n5)

performance on test data from gpt-4 with a default config and n=5: {'expected_success': 0.9197771144278607, 'success': 0.9552238805970149, 'success_vote': 0.8656716417910447, 'votes': 3.691542288557214, 'cost': 7.518809999999999, 'inference_cost': 0.03740701492537313}


We find that the 'success_vote' metric is increased at the cost of exceeding the inference budget. But the tuned configuration has both higher 'success_vote' (91% vs. 87%) and lower average inference cost ($0.015 vs. $0.037 per instance).

A developer could use flaml to tune the configuration to satisfy the target inference budget while maximizing the value out of it.